Notebook for implementing DMRG tree structure optimization on tensor network level. 

In [1]:
# Add in the neccesary libraries and depedencies
# First precompile takes more time -> Julia creates a underlying type-structure so that it can exploit the JIT feature
using Plots
using Makie
using GraphMakie.NetworkLayout
using CairoMakie
using Tenet
using TensorOperations
using LinearAlgebra
using Graphs
using GraphPlot
using EinExprs
using Combinatorics
using LaTeXStrings
Makie.inline!(true)
include("julia_functions.jl")



In [ ]:
# Implement a frucht graph TN, try to selected a cycle to transform into DMRG approachable tree

dimension = 2
G = Graphs.smallgraph(:frucht)
tn1 = fill_with_random(G, dimension, false, true)
locs_x =     [4, 4, -5, -2, 0, 0, 2, 0, -3, -1, -6, -4]
locs_y = -1*[-2, 1, -2, -1, 0, -2, 0, 3, 3, 1, 1, 0]
g, tensordict, edgesdict, fully_weighted_edge_list, edge_index_map = extract_graph_representation(tn1, false)

drawing1 = Makie.plot(tn1, node_color=[:darkred for i in 1:length(tensors(tn1))], labels=true, layout=Stress(), edge_color=:grey80)
display(drawing1)


function replace_index(tensor, replace_inds, new_inds)
    
    mapping = Dict(zip(replace_inds, new_inds))
    #display(mapping)
    current_inds = inds(tensor)
    ids = []
    for id in current_inds
        if id in replace_inds
            push!(ids, mapping[id])
        else
            push!(ids, id)
        end
    end
    new_tensor = Tenet.Tensor(tensor.data, [ids...])
    return new_tensor
end


function extract_partial_tn_loop(TN, cycle)
    tensors_in_loop = collect(Set([tensor for id in cycle for tensor in Tenet.select(TN, Symbol(id))])) #correct orderding of variable definitions
    current_tn = TensorNetwork(tensor for tensor in tensors_in_loop)
    #drawing1 = Makie.plot(current_tn, node_color=[:darkred for i in 1:length(tensors(current_tn))], labels=true, layout=Stress(), edge_color=:grey80)
    #display(drawing1)
    index_cut = Symbol(cycle[1])
    edge_tensors = Tenet.select(TN, index_cut)

    return current_tn
end



function generate_unique_symbols(existing_indices::Vector{Symbol}, l::Int, min_value::Int=1, max_value::Int=200)

    """
    Function which generates a list of new symbols for index replacing capability
    """

    existing_integers = [parse(Int, string(id)) for id in existing_indices]
    potential_length = round(Int, 4*l)  # Generate at least twice the desired length
    new_symbols = []
    for i in 1:potential_length
        number = rand(min_value:max_value)
        if !in(number, existing_integers)
            if !in(number, new_symbols)
                push!(new_symbols, number)
            end
        end
        if length(new_symbols) == l
            new_symbols = [Symbol(new) for new in new_symbols]
            break
        end
    end


  
    return new_symbols
  
end

function collect_tensors_along_loop(loop_tn, start_tensor, end_tensor, index_cycle, edge_cut)

    """
    Funtion whcih sequentially walks along the loop and selects the relevant tensors
    """

    tensors_ordered_loop = []
    index_cycle = filter(x -> x != edge_cut, index_cycle)
    for i in 1:length(tensors(loop_tn))
        if i == length(tensors(loop_tn))
            push!(tensors_ordered_loop, end_tensor)
            return tensors_ordered_loop
        end
        if i == 1
            push!(tensors_ordered_loop, start_tensor)
            continue
        end
        connecting_id = intersect(inds(tensors_ordered_loop[i-1]), [Symbol(id) for id in index_cycle])
        next_tensor = [tensorc for tensorc in Tenet.select(loop_tn, connecting_id)  if tensorc != tensors_ordered_loop[i-1]][1]
        index_cycle = filter(x -> x != parse(Int, string(connecting_id[1])), index_cycle)
        push!(tensors_ordered_loop, next_tensor)
    end
end


function create_index_isomorphismgroup(size_index1, size_index2, indices)
    n = size_index1*size_index2
    identity_data = Matrix{Int64}(I, n, n)
    tensor_isomorphism = reshape(identity_data, (size_index1, size_index2, n))
    #println("A grouping has been created: ", indices[1:2], "==>", indices[3])
    identity_isomorphism = Tenet.Tensor(tensor_isomorphism, [(indices...)])
    return identity_isomorphism
end


function create_index_isomorphismsplit(propagation_size, cut_size, new_indices, propagation_index)
    size_id_to_tensor = convert(Int64, propagation_size/cut_size)
    size_loop_part = cut_size
    n = propagation_size
    identity_data = Matrix{Int64}(I, n, n)
    tensor_isomorphism = reshape(identity_data, (propagating_size, size_loop_part, size_id_to_tensor))
    #println("A splitting has been created: ", propagation_index, "==>", new_indices[1:2])
    identity_isomorphism = Tenet.Tensor(tensor_isomorphism, [propagating_index, new_indices...])
    return identity_isomorphism
end


function transform_to_MPS(TN, edge_to_cut, index_cycle, printing=false)

    """
    Function which takes in a tensor network, a cycle of indices
    and an edge to cut, returns the MPS like structured TensorNetwork instead
    of the loop TensorNetwork.
    """
    



    inds_in_use = inds(TN)
    loop_tn = extract_partial_tn_loop(TN, index_cycle)
    n = length(tensors(loop_tn))
    
    # Generate an adequate amount of new indices to be used inside of the
    # virtual tensor structure
    new_inds = generate_unique_symbols(inds_in_use, 4*n-3)
    start_tensor, end_tensor = Tenet.select(TN, Symbol(edge_to_cut))

    #collect tensors along the loop from start_tensor to end_tensor
    ordered_along_loop = collect_tensors_along_loop(loop_tn, start_tensor, end_tensor, deepcopy(index_cycle), edge_to_cut)
    
    # list for the new tensors inside of the tensor network
    new_tn_tensors = []
    global contract_list = []
    global propagating_size
    global cut_size
    global propagating_index

    # walk along the loop and make the necessary alterations at each step
    for (i, tensor) in enumerate(ordered_along_loop)



        # cases on the edges of the tensor network loop are treated seperately
        if i == 1
            # initial grouping and determine the cut_
            # size of the index which is being collapsed

            cut_size = size(tensor, Symbol(edge_to_cut))
            new_virtual_inds = deepcopy(new_inds[1:3])
            current_inds = inds(tensor)
            dangling_leg = setdiff(inds(tensor), [Symbol(id) for id in index_cycle])
            replace_leg = intersect(inds(tensor), [Symbol(id) for id in index_cycle])
            iso = create_index_isomorphismgroup(size(tensor, replace_leg[1]), size(tensor, replace_leg[2]), new_virtual_inds)            
            new_tensor = replace_index(tensor, replace_leg, new_virtual_inds[1:2])
            contract_list = vcat(contract_list, new_virtual_inds[1:2])
            push!(new_tn_tensors, new_tensor)

            if printing == true
                println("indices tensor $(i)", inds(new_tensor), "with sizes", [size(new_tensor, id) for id in inds(new_tensor)])
                println("indices identity_1:", inds(iso), "with sizes", [size(iso, id) for id in inds(iso)])
            end

            push!(new_tn_tensors, iso)
            propagating_index = inds(iso)[end]
            propagating_size = size(iso, propagating_index)
            continue
        end

        if i == length(ordered_along_loop)
            
            new_virtual_inds = deepcopy(new_inds[end-1:end])
            current_inds = inds(tensor)
            dangling_leg = setdiff(inds(tensor), [Symbol(id) for id in index_cycle])
            replace_leg = intersect(inds(tensor), [Symbol(id) for id in index_cycle])
            iso_inv = create_index_isomorphismsplit(propagating_size, cut_size, new_virtual_inds, propagating_index)
            new_tensor = replace_index(tensor, replace_leg, new_virtual_inds)
            if printing == true
                println("indices identity_1_inv $(i-1)", inds(iso_inv), "with sizes", [size(iso_inv, id) for id in inds(iso_inv)])
                println("indices tensor $(i)", inds(new_tensor), "with sizes", [size(new_tensor, id) for id in inds(new_tensor)])
            end
            contract_list = vcat(contract_list, new_virtual_inds[1:2])
            push!(new_tn_tensors, new_tensor)
            push!(new_tn_tensors, iso_inv)
            break
        end
        
        # General "bulk" case
        new_virtual_inds = new_inds[4+(i-2)*4:4+(i-1)*4-1]
        iso_inv = create_index_isomorphismsplit(propagating_size, cut_size, [new_virtual_inds[1], new_virtual_inds[2]], propagating_index)
        current_inds = inds(tensor)
        dangling_leg = setdiff(inds(tensor), [Symbol(id) for id in index_cycle])
        replace_leg = intersect(inds(tensor), [Symbol(id) for id in index_cycle])
        new_tensor = replace_index(tensor, replace_leg, [new_virtual_inds[1], new_virtual_inds[3]])
        iso = create_index_isomorphismgroup(cut_size, size(new_tensor, new_virtual_inds[3]), [new_virtual_inds[2], new_virtual_inds[3], new_virtual_inds[4]])
        propagating_index = inds(iso)[end]
        propagating_size = size(iso, propagating_index)

        if printing == true
            println("indices identity_1_inv $(i-1)", inds(iso_inv), "with sizes", [size(iso_inv, id) for id in inds(iso_inv)])
            println("indices tensor $(i)", inds(new_tensor), "with sizes", [size(new_tensor, id) for id in inds(new_tensor)])
            println("indices identity $(i)", inds(iso), "with sizes", [size(iso, id) for id in inds(iso)])
        end

        contract_list = vcat(contract_list, new_virtual_inds[1:3])

        push!(new_tn_tensors, iso_inv)
        push!(new_tn_tensors, new_tensor)
        push!(new_tn_tensors, iso)


    end

    return Tenet.TensorNetwork(new_tn_tensors), contract_list

end


example_cycle = [9, 8, 11]

mps_network, contract_list = transform_to_MPS(tn1, example_cycle[1], example_cycle)
loop_tn = extract_partial_tn_loop(tn1, example_cycle)
drawing1 = Makie.plot(loop_tn, node_color=[:darkred for i in 1:length(tensors(loop_tn))], labels=true, layout=Stress(), edge_color=:grey80)
display(drawing1)

drawing1 = Makie.plot(mps_network, node_color=[:darkred for i in 1:length(tensors(mps_network))], labels=true, layout=Stress(), edge_color=:grey80)
display(drawing1)



for id in contract_list
    #contracting two tensors contracts fully along all shared indices
    # --> not everything in the list will be in the tensornetwork
    if id ∈ inds(mps_network)
        contraction_step(mps_network, [id])
    end
   
end
drawing1 = Makie.plot(mps_network, node_color=[:darkred for i in 1:length(tensors(mps_network))], labels=true, layout=Stress(), edge_color=:grey80)
display(drawing1)

### Comparing loop TN with MPS TN
contract_list_loop = inds(loop_tn, :inner)


for id in contract_list_loop
    if id ∈ inds(loop_tn)
        contraction_step(loop_tn, [id])
    end
end



contract_list_MPS = inds(mps_network, :inner)

for id in contract_list_MPS
    if id ∈ inds(mps_network)
        contraction_step(mps_network, [id])
    end
end


t1 = tensors(loop_tn)[1]
t2 = tensors(mps_network)[1]

println("original loop contracted")
println(t1[1,1,1])
println("mps representation contracted")
println(t2[1,1,1])






In [ ]:
function convert_TenetMPS_to_MPSkit(TenetMPS)
    